In [1]:
import pandas as pd
import implicit
import os
import numpy as np
from scipy import sparse

Решается задача товарных рекомендаций.

- `purchases_train.csv` - история покупок в розничном магазине (с 21 октября 2003 года по 12 марта 2004 года)

- `purchases_test.csv` - покупки за следующую неделю (с 13 по 19 марта 2004 года). В этой выборке для каждого пользователя исключены товары, которые он уже покупал за период обучающей выборки

- `customers.csv` - пол клиентов

В решении ниже
- обучается модель матричного разложения AlternateLeastSquares
- сравнивается с тестовыми данными
- измеряется ее качество по метрике map@10
- данные о поле клиентов не используются

Улучшите решение с помощью информацию о поле клиентов

In [2]:
purchases_train = pd.read_csv('purchases_train.csv')
purchases_train.head()

,customer_id,product_id,datetime
0,8698595,12530,2004-03-10 22:18:43.497459200
1,13271885,7541,2004-03-06 02:24:43.209763200
2,16852746,13134,2004-03-10 01:03:09.598614400
3,16852746,6572,2004-03-04 16:45:16.522566400
4,14619070,4659,2004-03-12 13:29:35.011481600


## Базовое решение - уменьшение размерности

<span style='color:navy'>Базовое решение использует customer_id и product_id как int-переменные. Это увеличивает размерность sparse-матрицы. Закодируем все уникальные значения клиентов и продуктов как натуральные ряды длиной в размер листов уникальных значений.</span>

In [3]:
purchases_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 351686 entries, 0 to 351685
Data columns (total 3 columns):
customer_id    351686 non-null int64
product_id     351686 non-null int64
datetime       351686 non-null object
dtypes: int64(2), object(1)
memory usage: 8.0+ MB


In [4]:
# Уникальные значения клиентов и продуктов
unique_customers = purchases_train.customer_id.unique().tolist()
unique_products = purchases_train.product_id.unique().tolist()
# Словари для энкодинга и декодинга
customer_encode_dict = {unique_customers[i]:i for i in range(len(unique_customers))}
customer_decode_dict = {i:unique_customers[i] for i in range(len(unique_customers))}
product_encode_dict = {unique_products[i]:i for i in range(len(unique_products))}
product_decode_dict = {i:unique_products[i] for i in range(len(unique_products))}

In [5]:
purchases_train_encoded = purchases_train.copy()
purchases_train_encoded['customer_id'] = purchases_train_encoded['customer_id'].map(customer_encode_dict)
purchases_train_encoded['product_id'] = purchases_train_encoded['product_id'].map(product_encode_dict)

Обучаем модель AlternateLeastSquares

In [6]:
user_items = sparse.coo_matrix(
    (
        np.ones(purchases_train_encoded.customer_id.size, dtype=np.float32),
        (
            purchases_train_encoded.customer_id,
            purchases_train_encoded.product_id
        )
    )
).tocsr()

In [7]:
item_users = user_items.T.tocsr()

In [8]:
os.environ['MKL_NUM_THREADS'] = '1'

In [9]:
model = implicit.als.AlternatingLeastSquares(factors=64,
                                             iterations=100,
                                             num_threads=10)

In [10]:
%%time
np.random.seed(42)
model.fit(item_users=item_users)


CPU times: user 12min 10s, sys: 18min 46s, total: 30min 56s
Wall time: 2min 36s


purchases_test.csv содержит данные о покупках с 13 марта 2004 по 20 марта 2004 - то есть неделя следующая за обучающей выборкой

для каждого пользователя исключены те товары, которые он покупал в обучающей выборке

In [11]:
purchases_test = pd.read_csv('purchases_test.csv')
display(
    purchases_test.head(),
)

,customer_id,product_id,datetime
0,1021292,6197,2004-03-18 13:35:19.145152000
1,11379978,4659,2004-03-19 18:51:31.887936000
2,13271885,5659,2004-03-14 05:47:21.544166400
3,13271885,1015,2004-03-15 14:41:19.702089601
4,12315337,12072,2004-03-19 10:39:17.148105600


In [12]:
purchases_test[~purchases_test.customer_id.isin(unique_customers)].shape

(0, 3)

In [13]:
purchases_test[~purchases_test.product_id.isin(unique_products)].shape

(7, 3)

<span style='color:navy'>Видим, что в тестовой выборке добавились новые продукты.</span>

In [14]:
# Запоминаем продукты, которых не было в трейне
not_in_train = purchases_test[~purchases_test.product_id.isin(unique_products)].product_id.unique().tolist()

In [15]:
# Обновляем словари для продуктов
product_encode_dict_test = {**product_encode_dict,\
                            **{not_in_train[i]:i+len(product_encode_dict) for i in range(len(not_in_train))}}
product_decode_dict_test = {v:k for k,v in product_encode_dict_test.items()}

In [16]:
purchases_test_encoded = purchases_test.copy()
purchases_test_encoded['customer_id'] = purchases_test_encoded['customer_id'].map(customer_encode_dict)
purchases_test_encoded['product_id'] = purchases_test_encoded['product_id'].map(product_encode_dict_test)

Измеряем качество рекомендаций с помощью метрики map@10

In [17]:
relevant = purchases_test_encoded.groupby('customer_id')['product_id'].apply(lambda s: s.values).reset_index()
relevant.rename(columns={'product_id': 'product_ids'}, inplace=True)
relevant.head()

,customer_id,product_ids
0,0,[5]
1,1,"[191, 669, 282, 1190, 200, 856, 173, 1091, 10,..."
2,2,"[65, 505, 492, 808]"
3,3,"[245, 333, 147, 92, 404, 336]"
4,6,"[282, 10, 331, 791]"


In [18]:
recommendations = []
for user_id in relevant['customer_id']:
    recommendations.append([x[0] for x in model.recommend(userid=user_id, user_items=user_items, N=10)])

In [19]:
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if len(actual) == 0:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [20]:
mapk(relevant['product_ids'], recommendations, k=10)

0.10047918492023387

<span style='color:navy'>Кодирование таблицы слегка уменьшило значение метрики, зато убыстрило процесс расчета алгоритма.</span>

## Улучшение качества модели

Задание: используйте пол клиентов для улучшения модели

In [21]:
customers = pd.read_csv('customers.csv')
customers.head()

,customer_id,sex
0,14386819,Female
1,1481405,NaN
2,16745074,NaN
3,10325906,NaN
4,11167384,NaN


In [22]:
customers = customers[customers.customer_id.isin(unique_customers)]

In [23]:
customers.sex.value_counts()

Female    23536
Male       8407
Name: sex, dtype: int64

<span style='color:navy'>Для того, чтобы использовать информацию о поле клиентов в implicit-постановке задачи, закодируем ее с помощью one-hot encoding и присоединим к исходной матрице товаров. Таким образом, похожесть пользователей будет определяться не только товарами, но и полом. Подобный подход также описан в ссылке https://developers.google.com/machine-learning/recommendation/collaborative/summary.</span>

In [24]:
used_customers = customers.copy()
used_customers['customer_id'] = used_customers['customer_id'].map(customer_encode_dict)

In [25]:
used_customers = used_customers.join(pd.get_dummies(used_customers.sex))\
                               .drop('sex', axis=1)\
                               .sort_values(by='customer_id')\
                               .set_index('customer_id')

In [26]:
user_items.shape

(107491, 1792)

In [27]:
# Не забываем между товарами из трейна и атрибутами полов вставить товары из теста, дабы не было ошибок
user_items_sex = sparse.hstack([user_items,\
                                np.zeros((user_items.shape[0], len(not_in_train)), dtype=np.float32),\
                                used_customers]).tocsr()

In [28]:
item_users_sex = user_items_sex.T.tocsr()

In [29]:
model_sex = implicit.als.AlternatingLeastSquares(factors=64,
                                                 iterations=100,
                                                 num_threads=10)

In [30]:
%%time
np.random.seed(42)
model_sex.fit(item_users=item_users_sex)


CPU times: user 12min 5s, sys: 18min 49s, total: 30min 55s
Wall time: 2min 36s


In [31]:
recommendations_sex = []
for user_id in relevant['customer_id']:
    recommendations_sex.append([x[0] for x in model_sex.recommend(userid=user_id, user_items=user_items_sex, N=10)])

In [32]:
mapk(relevant['product_ids'], recommendations_sex, k=10)

0.11133337442779122

<span style='color:navy'>Добавление пола клиентов улучшило исходное значение метрики map@10 (0.104). Для промышленного использования рекомендаций мы можем декодировать значения клиентов и продуктов. На примере pandas-датафрейма это можно сделать так:</span>

In [33]:
df_recommendations = pd.DataFrame(recommendations_sex, columns=[f'choice_{i}' for i in range(10)])

In [34]:
df_recommendations.index = relevant.customer_id.map(customer_decode_dict)
df_recommendations = df_recommendations.replace(product_decode_dict_test)

In [35]:
df_recommendations.head()

,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5,choice_6,choice_7,choice_8,choice_9
customer_id,,,,,,,,,,
8698595,7240,11131,8621,2984,5327,11570,61,2683,2842,336
13271885,220,27,10913,7754,5220,12846,3770,9132,338,9270
16852746,11131,11854,4623,336,12667,9350,11338,10913,3064,3020
14619070,13690,5726,11898,9596,9240,12074,13529,7909,7754,702
343816,10913,11898,3770,5220,12846,9132,7470,27,3419,9350
